# 모폴로지 팽창 연산 (morphology dilation operation)
- 팽창 연산은 객체를 팽창시키는 연산이다. 
- 객체의 가장 외곽 화소를 확상지시키 대문에 객체의 크기는 확대되고 배경은 축소된다. 또한, 객체의 팽창으로 인해서 객체 내부에 있는 빈 공간도 매워지게 된다. 

![dilate](img/dilation.png)

- 이진 영상에서 팽창 연산을 수행하는 과정을 나나낸 거싱며, 마스크 범위의 입력 화소와 팽창 마스크의 원소 간 (element-wise)에 일치하는지를 비교한다. 팽창 마스크가 1인 원소와 해당 입력 화소가 모두 불일치하면 출력 화소로 0을 지정한다. 그리고 1개 화소라도 일치하게 되며 1이 출력 화소로 결정된다. 

In [1]:
import numpy as np, cv2

In [2]:
def dilate(img, mask):
    dst = np.zeros(img.shape, np.uint8)
    if mask is None: mask = np.ones((3, 3), np.uint8)
    ycenter, xcenter = np.divmod(mask.shape[:2], 2)[0]

    for i in range(ycenter, img.shape[0] - ycenter):    # 입력 행렬 반복 순회
        for j in range(xcenter, img.shape[1] - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1       # 마스크 높이 범위
            x1, x2 = j - xcenter, j + xcenter + 1       # 마스크 너비 범위
            roi = img[y1:y2, x1:x2]                     # 마스크 영역
            temp = cv2.bitwise_and(roi, mask)
            cnt  = cv2.countNonZero(temp)
            dst[i, j] = 0 if (cnt == 0) else 255  # 출력 화소에 저장
    return dst

In [3]:
image = cv2.imread("img/morph.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상파일 읽기 오류")

In [4]:
mask = np.array([[0, 1, 0],                         # 마스크 초기화
                 [1, 1, 1],
                 [0, 1, 0]]).astype("uint8")
th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]  # 영상 이진화
dst1 = dilate(th_img, mask)                              # 사용자 정의 팽창 함수
dst2 = cv2.morphologyEx(th_img, cv2.MORPH_DILATE, mask)  # OpenCV의 팽창 함수
# dst2 = cv2.dilate(th_img, mask)

In [5]:
cv2.imshow("User dilate", dst1)
cv2.imshow("OpenCV dilate", dst2)
cv2.waitKey(0)

-1

![result](img/14_result.png)

- 팽창 연산의 수행 결과로 객체들이 전반적으로 확장되어 뚜렷하게 나타나고, 객체 내부의 빈 공간이 매워지게 된다. 다만, 배경 부분의 잡음들도 확장된 것을 볼 수 있다. 